In [1]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'none'

# seaborn package for making pretty plots, but not necessary
try:
    import seaborn as sns
    params =   {'xtick.direction': 'in', 'ytick.direction': 'in', 'font.family': ['serif'],
                'text.usetex': True, 'text.latex.preamble': ['\usepackage{gensymb}']}
    sns.set_style("ticks", params)
except ImportError:
    print('Seaborn module is not installed.')
    
from IPython.display import display, Latex, clear_output

In [2]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import (webbpsf, poppy, pix_noise, S)

pynrc.setup_logging('WARNING', verbose=False)

from astropy.table import Table

In [3]:
# Initiate a NIRCam observation
nrc = pynrc.NIRCam('F444W', pupil='GRISM0', wind_mode='STRIPE', ypix=64)

# We want to optimize SNR of a K=16 M2V star
bp_k = S.ObsBandpass('johnson,k')
sp_M2V = pynrc.stellar_spectrum('M2V', 10, 'vegamag', bp_k)

nrc.update_detectors(read_mode='DEEP2', nint=35, ngroup=5)
snr1=nrc.sensitivity(sp=sp_M2V, forwardSNR=True, verbose=True)
nrc.update_detectors(read_mode='MEDIUM2', nint=35, ngroup=9)
snr2=nrc.sensitivity(sp=sp_M2V, forwardSNR=True, verbose=True)

#_ = nrc.sat_limits(verbose=True)
#_ = nrc.sensitivity(verbose=True, units='vegamag')

F444W SNR for M2V source:
Wave       SNR      Flux (uJy)
3.90   1111.09  29767.23
4.00   1169.43  28361.64
4.10   1121.30  27049.16
4.20   1071.35  26158.33
4.30    978.24  23226.08
4.40    922.63  22468.60
4.50    845.03  20824.46
4.60    741.44  17733.72
4.70    679.41  17170.92
4.80    608.38  15411.13
4.90    622.94  19485.87
5.00    344.29  17055.91
F444W SNR for M2V source:
Wave       SNR      Flux (uJy)
3.90   1114.49  29767.23
4.00   1172.67  28361.64
4.10   1124.67  27049.16
4.20   1074.86  26158.33
4.30    982.04  23226.08
4.40    926.63  22468.60
4.50    849.33  20824.46
4.60    746.21  17733.72
4.70    684.51  17170.92
4.80    613.90  15411.13
4.90    628.36  19485.87
5.00    351.72  17055.91


In [4]:
# Initiate a NIRCam observation
nrc = pynrc.NIRCam('F444W', pupil=None, wind_mode='STRIPE', ypix=64)
#print(nrc.sat_limits())

# We want to optimize SNR of a K=16 M2V star
bp_k = S.ObsBandpass('johnson,k')
sp_M2V = pynrc.stellar_spectrum('M2V', 15, 'vegamag', bp_k)

In [5]:
t = nrc.ramp_optimize(sp_M2V, tacq_max=1000, verbose=True, return_full_table=True, ng_min=5, snr_frac=0, \
                     well_frac_max=0.5)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
Top 10 results sorted by 'efficiency' (SNR/t_acq):
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2      10   32     31.34   1002.76   1013.66   2501.7    0.465   78.574
MEDIUM2       9   35     27.93    977.55    989.47   2465.9    0.415   78.390
DEEP2         5   36     27.93   1005.48   1017.74   2497.9    0.415   78.298
MEDIUM2       8   41     24.52   1005.48   1019.45   2495.2    0.364   78.148
MEDIUM2       7   47     21.12    992.54   1008.55   2471.3    0.313   77.817
MEDIUM2       6   57     17.71   1009.57   1028.98   2481.1    0.263   77.347
SHALLOW2     10   63     16.01   1008.55   1030.00   2477.4    0.238   77.191
SHALLOW2      9   71     14.31   1015.70   1039.88   2477.1    0.212   76.815
MEDIUM2       5   71     14.31   1015.70   1039.88   2471.3    0.212   76.636
SHALLOW2      8   81   

In [12]:
print(t[t['Pattern']=='MEDIUM2'])
print(t[t['Pattern']=='DEEP2'])

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2       5   71     14.31   1015.70   1039.88   2435.7    0.212   75.532
MEDIUM2       6   57     17.71   1009.57   1028.98   2422.1    0.263   75.507
MEDIUM2       7   49     21.12   1034.77   1051.46   2444.7    0.313   75.392
MEDIUM2       8   42     24.52   1030.00   1044.31   2431.8    0.364   75.250
MEDIUM2       9   37     27.93   1033.41   1046.01   2429.1    0.415   75.106
MEDIUM2      10   33     31.34   1034.09   1045.33   2423.9    0.465   74.968
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
DEEP2         5   35     27.93    977.55    989.47   2421.3    0.415   76.975
DEEP2         6   29     34.74   1007.52   1017.40   2444.3    0.516   76.631
DEEP2         7   25     41.55   1038.86   1047.38   2469.1    0